## Setting up

In [1]:
#Loading packages

using Pkg
#Pkg.add("EcologicalNetworks")

#import Pkg; Pkg.add("CSV"); Pkg.add("DataFrames")
using CSV
using DataFrames
using EcologicalNetworks
using EcologicalNetworksPlots



In [29]:
# Setting up functions

#This function opens and reads all the files
function openwebs(webnames)
    dfs = []

    for web in webnames
        filename = "../data/" * web * "_adjacency_matrix.csv"

        df = CSV.read(filename,DataFrame)

        # Append the df to the list
        push!(dfs, df)

    end

    return dfs

end

openwebs (generic function with 1 method)

In [3]:
# This extracts the species names from the files
function getspecies(webs)
    Ss = []

    for web in webs
        # Get the first column which has the names of all the species
        S = web.Column1

        # Append S to the list
        push!(Ss, S)

    end

    return Ss

end

getspecies (generic function with 1 method)

In [30]:
# Convert webs to boolean array
function convertWebsToBoolArray(webs)
    As = []

    for web in webs
        ## Remove the column with the names
        A = Array(web[:,2:end])
        A = transpose(A)

        # Convert to boolean
        A = isone.(A)

        # Append A to the list
        push!(As, A)

    end

    return As

end

convertWebsToBoolArray (generic function with 1 method)

In [5]:
# Convert the output into an array with species and the results for each web
function create_dict_array(dicts, webnames, Ss)
    species = collect(keys(dicts[1]))

    for d in dicts[2:end]
        species = union(species, keys(d))
    end
    
    #data = Array[length(species) + 1, length(dicts)]
    data = Array{Any}(undef, length(species)+1, length(dicts) + 1)

    data[1, 1] = "species"

    for i in 2:length(species)+1
        data[i, 1] = species[i-1]
    end

    for j in 1:length(dicts)
        for i in 2:length(species)+1
            data[i, j+1] = get(dicts[j], species[i-1], "")
        end
    end
    
    #df = DataFrame(data, :auto)
    data[1,2:4] = webnames

    return data
end

create_dict_array (generic function with 1 method)

In [6]:
function get_modularity(N, output = "all")
    n = repeat(3:12, outer=20)
    m = Array{Dict}(undef, length(n))

    for i in eachindex(n)
      # Each run returns the network and its modules
      # We discard the network, and assign the modules to our object
      _, m[i] = n_random_modules(n[i])(Ns[1]) |> x -> brim(x...)
    end

    q = map(x -> Q(Ns[1],x), m);
    c = (m .|> values |> collect) .|> unique .|> length


    #using Plots
    #using Plots.PlotMeasures

    #p1 = scatter(c, q, c=:grey, msw=0.0, leg=false, frame=:origin, grid=false, margin=10mm)
    #xaxis!(p1, "Number of modules")
    #yaxis!(p1, "Modularity", (0, 0.1))
    
    optimal = rand(findall(q.== maximum(q)));
    best_m = m[optimal]
    modularity = maximum(q)
    number_modules = c[optimal]
        
    
    if output == "all"
        return number_modules, modularity, best_m
    end
    if output == "number_modules"
        return number_modules
    end
    if output == "modularity"
        return modularity
    end
    if output == "modules"
        return best_m
    end
end

get_modularity (generic function with 2 methods)

In [7]:
function plot_modules(modules_species)

    I = initial(RandomInitialLayout, Ns[1])
    for step in 1:4000
      position!(SpringElectric(1.2; gravity=0.1), I, Ns[1])
    end
    p2 = plot(I, Ns[1], aspectratio=1)
    scatter!(p2, I, Ns[1], bipartite=true, nodefill=best_m, markercolor=:isolum)

end

plot_modules (generic function with 1 method)

## Our data

In [28]:
webnames = ["Premonsoon","Postmonsoon","Monsoon"]
webs = openwebs(webnames)


LoadError: ArgumentError: `transpose` not defined for `AbstractDataFrame`s. Try `permutedims` instead

In [13]:
# Get lists of species
Ss = getspecies(webs)

3-element Vector{Any}:
 String31["Acridotheres ginginianus", "Alcedo atthis", "Alona pulchella", "Amaurornis phoenicurus", "Anas poecilorhyncha", "Anastomus oscitans", "Anisops campbelli", "Aplocheilus panchax", "Aquarius adelaides", "Arcella discoides"  …  "Rasbora daniconius", "Scapholeberis kingi", "Spilornis cheela", "Tachybaptus ruficollis", "Trichogaster fasciata", "Trichogaster lalius", "Uperodon systoma", "Vanellus indicus", "Varanus bengalensis", "Xenentodon cancila"]
 String31["Acridotheres ginginianus", "Alcedo atthis", "Alona pulchella", "Amaurornis phoenicurus", "Anas acuta", "Anas poecilorhyncha", "Anastomus oscitans", "Anhinga melanogaster", "Anisops campbelli", "Aplocheilus panchax"  …  "Tringa glareola", "Tringa nebularia", "Tringa ochropus", "Tringa stagnatilis", "Tringa tetanus", "Vanellus indicus", "Vanellus malabaricus", "Varanus bengalensis", "Xenentodon cancila", "Xenus cinereus"]
 String31["Acridotheres ginginianus", "Alcedo atthis", "Alona pulchella", "Amaurorn

In [31]:
# Convert networks to boolean arrays, then to unipartite networks
As = convertWebsToBoolArray(webs)
Ns = [UnipartiteNetwork(As[j],Ss[j]) for j in 1:length(As)]


3-element Vector{UnipartiteNetwork{Bool, String31}}:
 109×109 (String31) unipartite ecological network (L: 1638 - Bool)
 135×135 (String31) unipartite ecological network (L: 1924 - Bool)
 111×111 (String31) unipartite ecological network (L: 1717 - Bool)

## Network level analysis

In [32]:
# How many links does each network have?
links_byweb = [links(N) for N in Ns]


3-element Vector{Int64}:
 1638
 1924
 1717

In [33]:
# What's the connectance of each network?
C_byweb = [connectance(N) for N in Ns]


3-element Vector{Float64}:
 0.13786718289706254
 0.10556927297668038
 0.13935557178800423

In [34]:
# What's the modularity of each network?
modular_info = [get_modularity(N) for N in Ns]
number_modules_byweb = [mod_info[1] for mod_info in modular_info]

3-element Vector{Int64}:
 3
 3
 3

In [35]:
modularity_byweb = [mod_info[2] for mod_info in modular_info]

3-element Vector{Float64}:
 0.1479547111415229
 0.15236537306134285
 0.1557801512013958

In [36]:
nestedness_byweb = [ρ(N) for N in Ns]

3-element Vector{Float64}:
 0.9676177161061251
 0.9502405689149828
 0.9671215976142671

## Species level metrics

In [37]:
degrees_byweb = [EcologicalNetworks.degree(N) for N in Ns]
degree_array = create_dict_array(degrees_byweb, webnames, Ss)
### Write to csv
#CSV.write("data/Tampara/processed/degree_dataframe.csv",  Tables.table(degree_array), writeheader=false)

145×4 Matrix{Any}:
 "species"                              …    "Postmonsoon"    "Monsoon"
 "Scapholeberis kingi"           16               16
 "Nettapus coromandelianus"      16                 ""
 "Felis chaus"                    9               12
 "Centropus bengalensis"         17               25
 "Himantopus himantopus"      …  11               11
 "Euphlyctis cyanophlyctis"      34               31
 "Anas poecilorhyncha"           12                 ""
 "Alona pulchella"               15               15
 "Mastacembelus armatus"         33               31
 "Asplanchna brightwelli"     …  15               15
 "Glossogobius giuris"           35               34
 "Polypedates maculatus"         36               32
 ⋮                                      ⋱                   
 "Microhyla mymensinghensis"     37               34
 "Phalacrocorax fuscicollis"      9                 ""
 "Charadrius dubius"          …  11                 ""
 "Motacilla tschutschensis"      11          

In [38]:
outdegrees_byweb = [EcologicalNetworks.degree(N, dims = 1) for N in Ns]
outdegree_array = create_dict_array(outdegrees_byweb, webnames, Ss)
### Write to csv
#CSV.write("data/Tampara/processed/outdegree_dataframe.csv",  Tables.table(outdegree_array), writeheader=false)


specificity_byweb = [specificity(N) for N in Ns]
specificity_array = create_dict_array(specificity_byweb, webnames, Ss)
### Write to csv
#CSV.write("data/Tampara/processed/specificity_dataframe.csv",  Tables.table(specificity_array), writeheader=false)


centrality_degree_byweb = [centrality_degree(N) for N in Ns]
centrality_degree_array = create_dict_array(centrality_degree_byweb, webnames, Ss)
### Write to csv
#CSV.write("data/Tampara/processed/centrality_degree_dataframe.csv",  Tables.table(centrality_degree_array), writeheader=false)


centrality_closeness_byweb = [centrality_closeness(N) for N in Ns]
centrality_closeness_array = create_dict_array(centrality_closeness_byweb, webnames, Ss)
### Write to csv
#CSV.write("data/Tampara/processed/centrality_closeness_dataframe.csv",  Tables.table(centrality_closeness_array), writeheader=false)


overlap_byweb = [overlap(N) for N in Ns]       # calculate overlap based on prey (dims = 1) or predators (dims = 2)
#overlap_array = create_dict_array(overlap_byweb, webnames)   # This is more complicated because its done by pairs of species

AJS_byweb = [AJS(N) for N in Ns]
#AJS_array = create_dict_array(AJS_byweb, webnames) # This is more complicated because its done by pairs of species

trophic_level_byweb = [trophic_level(N) for N in Ns]
trophic_level_array = create_dict_array(trophic_level_byweb, webnames, Ss)
### Write to csv
#CSV.write("data/Tampara/processed/trophic_level_dataframe.csv",  Tables.table(trophic_level_array), writeheader=false)


omnivory_byweb = [omnivory(N) for N in Ns]
omnivory_array = create_dict_array(omnivory_byweb, webnames, Ss)
### Write to csv
#CSV.write("data/Tampara/processed/omnivory_dataframe.csv",  Tables.table(omnivory_array), writeheader=false)

145×4 Matrix{Any}:
 "species"                              …    "Postmonsoon"    "Monsoon"
 "Scapholeberis kingi"            0.0              0.0
 "Nettapus coromandelianus"      10.2184            ""
 "Felis chaus"                    2.75403          4.07563
 "Centropus bengalensis"          7.49845         11.0111
 "Himantopus himantopus"      …   3.58017          3.58017
 "Euphlyctis cyanophlyctis"       3.58017          3.58017
 "Anas poecilorhyncha"            8.32108           ""
 "Alona pulchella"                0.0              0.0
 "Mastacembelus armatus"          8.10633          8.10633
 "Asplanchna brightwelli"     …   0.0              0.0
 "Glossogobius giuris"           19.2746          19.2746
 "Polypedates maculatus"          3.58017          3.58017
 ⋮                                      ⋱                   
 "Microhyla mymensinghensis"      3.58017          3.58017
 "Phalacrocorax fuscicollis"      2.3727            ""
 "Charadrius dubius"          …   3.58017       

In [42]:
modules_by_species = [mod_info[3] for mod_info in modular_info]
modules_array = create_dict_array(modules_by_species, webnames, Ss)
# I'm not sure these modules can be compared directly - we probably need to look more at the other species in the same module
#CSV.write("../data/Tampara/processed/modules_dataframe.csv",  Tables.table(modules_array), writeheader=false)


# Plot modules
#plot_modules(modules_by_species[2])

110×4 Matrix{Any}:
 "species"                               …   "Postmonsoon"   "Monsoon"
 "Scapholeberis kingi"            2               3
 "Nettapus coromandelianus"       3               1
 "Felis chaus"                    1               2
 "Centropus bengalensis"          3               1
 "Himantopus himantopus"       …  3               1
 "Euphlyctis cyanophlyctis"       3               1
 "Anas poecilorhyncha"            3               1
 "Alona pulchella"                2               3
 "Mastacembelus armatus"          3               1
 "Asplanchna brightwelli"      …  2               3
 "Glossogobius giuris"            3               1
 "Polypedates maculatus"          3               1
 ⋮                                       ⋱                  
 "Pethia phutunio"                2               3
 "Acridotheres ginginianus"       1               2
 "Fejervarya orissaensis"      …  3               1
 "Haliaeetus leucogaster"         1               2
 "Keratella lenzi

In [40]:
trophic_level_byweb = [trophic_level(N) for N in Ns]
trophic_level_array = create_dict_array(trophic_level_byweb, webnames, Ss)


145×4 Matrix{Any}:
 "species"                              …   "Postmonsoon"   "Monsoon"
 "Scapholeberis kingi"           2.0             2.0
 "Nettapus coromandelianus"      4.18867          ""
 "Felis chaus"                   4.87218         5.05064
 "Centropus bengalensis"         4.62872         4.87229
 "Himantopus himantopus"      …  4.27418         4.27418
 "Euphlyctis cyanophlyctis"      4.27418         4.27418
 "Anas poecilorhyncha"           4.08467          ""
 "Alona pulchella"               2.0             2.0
 "Mastacembelus armatus"         4.45957         4.45957
 "Asplanchna brightwelli"     …  2.0             2.0
 "Glossogobius giuris"           4.27037         4.27037
 "Polypedates maculatus"         4.27418         4.27418
 ⋮                                      ⋱                  
 "Microhyla mymensinghensis"     4.27418         4.27418
 "Phalacrocorax fuscicollis"     4.66788          ""
 "Charadrius dubius"          …  4.27418          ""
 "Motacilla tschutschens

In [43]:
# Need to figure this out, then can put all the data in one big data frame

modules_array_long = stack(DataFrame(modules_array, :auto), 2:4)

Row,x1,variable,value
,Any,String,Any
1,species,x2,Premonsoon
2,Scapholeberis kingi,x2,3
3,Nettapus coromandelianus,x2,1
4,Felis chaus,x2,3
5,Centropus bengalensis,x2,1
6,Himantopus himantopus,x2,1
7,Euphlyctis cyanophlyctis,x2,1
8,Anas poecilorhyncha,x2,1
9,Alona pulchella,x2,3


## Motifs


In [44]:
# Get all the motif tuples for each motif and each web
motif_lists = [find_motif(N,m) for m in unipartitemotifs(), N in Ns]



13×3 Matrix{Vector{Any}}:
 [([1, 7, 35],), ([1, 7, 36],), ([1, 7, 39],), ([1, 7, 40],), ([1, 7, 52],), ([1, 7, 53],), ([1, 7, 75],), ([1, 7, 76],), ([1, 7, 93],), ([1, 7, 106],)  …  ([98, 106, 108],), ([99, 101, 105],), ([99, 104, 105],), ([99, 105, 106],), ([99, 105, 107],), ([99, 105, 109],), ([99, 106, 108],), ([100, 101, 103],), ([101, 103, 104],), ([101, 103, 105],)]            …  [([1, 6, 34],), ([1, 6, 35],), ([1, 6, 38],), ([1, 6, 39],), ([1, 6, 50],), ([1, 6, 51],), ([1, 6, 53],), ([1, 6, 74],), ([1, 6, 75],), ([1, 6, 76],)  …  ([98, 105, 109],), ([98, 105, 111],), ([98, 106, 107],), ([98, 107, 108],), ([98, 107, 109],), ([98, 107, 111],), ([98, 108, 110],), ([99, 100, 104],), ([100, 104, 106],), ([100, 104, 107],)]
 [([1, 35, 53],), ([1, 36, 53],), ([1, 39, 52],), ([1, 39, 53],), ([1, 40, 52],), ([1, 40, 53],), ([1, 52, 53],), ([1, 52, 76],), ([1, 52, 106],), ([1, 53, 75],)  …  ([95, 97, 109],), ([95, 98, 105],), ([95, 98, 109],), ([95, 99, 104],), ([95, 99, 105],), ([95, 99,

In [ ]:
# motif 3 doesn't seem to exist in any of our webs, so this breaks if we do it normally
motifs = [1,2,4,5,6,7,8,9,10,11,12,13]

# Make a dataframe to store everything
species_motif_counts_df = DataFrame(web = String[], motif = Symbol[], species = String[], count = Int[])
# Get the motif names
motifnames = fieldnames(typeof(unipartitemotifs()))


# Run through each network, motif, and species in each network 
for N in 1:size(webnames,1)
    for m in motifs
        # Remember to use Ss[N] because some species don't occur in all networks
        for sp in 1:size(Ss[N],1)
            # get the name of the web, motif, and species
            web = webnames[N]
            motif = motifnames[m]
            species = Ss[N][sp]

            # Count how often that species appears in that motif
            # This could be edited to get each motif position, by adding a value in the final position of motif_lists. E.g. motif_lists[m,N][x][1,position]
            count = sum([sp in motif_lists[m,N][x][1,] for x in 1:size(motif_lists[m,N],1)]) 

            # Add a row to the dataframe
            species_motif_counts_df = push!(species_motif_counts_df, [web, motif, species, count])
        end
    end
end



CSV.write("data/Tampara/processed/species_motif_counts.csv", species_motif_counts_df, writeheader=FALSE)



In [ ]:
motif_list_premonsoon = [find_motif(Ns[1],m) for m in unipartitemotifs()]

In [ ]:
CSV.write("motif_list_premonsoon.csv", Tables.table(motif_list_premonsoon), writeheader=false)

In [ ]:
unipartitemotifs()



In [ ]:
S1_premonsoon = find_motif(Ns[1],unipartitemotifs()[1])

In [ ]:
CSV.write("S1_premonsoon.csv", Tables.table(S1_premonsoon), writeheader=false)
#CSV.write("FileName.csv",  Tables.table(A), writeheader=false)